### This code extracts the TTL times from the OneBox data streaming

In [1]:
import numpy as np

In [7]:
def read_meta(meta_path):
    meta = {}
    with open(meta_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('~'):
                continue
            if '=' in line:
                k, v = line.split('=', 1)
                meta[k] = v
    # Cast the fields we care about
    meta['nSavedChans'] = int(meta['nSavedChans'])
    meta['obSampRate'] = float(meta['obSampRate'])
    return meta

def load_obx_channel(bin_path, meta_path, chan_idx):
    meta = read_meta(meta_path)
    n_ch = meta['nSavedChans']
    fs = meta['obSampRate']

    # int16, little-endian (SpikeGLX / CatGT standard)
    raw = np.memmap(bin_path, dtype='<i2', mode='r')
    n_samples = raw.size // n_ch
    data = raw.reshape((n_samples, n_ch))

    ttl = data[:, chan_idx]
    return ttl, fs

def find_ttl_rising_edges(ttl, fs, threshold=None, min_interval=0.001):
    """
    ttl: 1D np.array of int16 or float
    fs:  sampling rate (Hz)
    threshold: value to define 'high' vs 'low'. If None, auto-estimate.
    min_interval: min time (s) between distinct TTL pulses (to de-bounce).
    """
    ttl = np.asarray(ttl)

    # Auto threshold: mid-point between low and high clusters
    if threshold is None:
        lo = np.percentile(ttl, 5)
        hi = np.percentile(ttl, 95)
        threshold = (lo + hi) / 2.0

    above = ttl >= threshold
    # Rising edges: was below, now above
    rising = np.where(~above[:-1] & above[1:])[0] + 1

    # Optional de-bounce: enforce minimum spacing between pulses
    if min_interval is not None and rising.size > 0:
        min_samples = int(min_interval * fs)
        keep = [rising[0]]
        for idx in rising[1:]:
            if idx - keep[-1] >= min_samples:
                keep.append(idx)
        rising = np.array(keep, dtype=int)

    times_sec = rising / fs
    return rising, times_sec, threshold

In [8]:
bin_path = r"\\research-cifs.nyumc.org\research\buzsakilab\Homes\voerom01\Bilat_HPC\Bilat_R02\Bilat_R02_20251106\preprocessing_output\supercat_pre_sleep_g0\pre_sleep_g0_tcat.obx0.obx.bin"
meta_path = r"\\research-cifs.nyumc.org\research\buzsakilab\Homes\voerom01\Bilat_HPC\Bilat_R02\Bilat_R02_20251106\preprocessing_output\supercat_pre_sleep_g0\pre_sleep_g0_tcat.obx0.obx.meta"

ttl, fs = load_obx_channel(bin_path, meta_path, chan_idx=6)
print(f'TTL shape: {ttl.shape}, FS: {fs}')

rising_samples, rising_times, thr = find_ttl_rising_edges(ttl, fs)

print(f"Detected {len(rising_samples)} TTL pulses")
print("First 10 times (s):", rising_times[:10])
print("Used threshold:", thr)

(318820441,) 30303.03025210084
Detected 308285 TTL pulses
First 10 times (s): [3926.3532066 3926.3615556 3926.3698716 3926.3782206 3926.3865366
 3926.3948856 3926.4032016 3926.4115506 3926.4198996 3926.4282156]
Used threshold: 10429.0
